In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import os 
import sys 
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.auto import tqdm
from data import *
from build_batches import load_battery_data

In [3]:
# load batch with custom function
bat_dict = load_battery_data('/Users/ivan_zorin/Documents/AIRI/data/batt/', is_save=False)

print(bat_dict['b1c0']['cycle_life'])

b1c0_cycles = [int(cycle) for cycle in bat_dict['b1c0']['cycles'].keys()]
print(max(b1c0_cycles))


In [ ]:
# load pickle file of gathered bathches directly with pickle

data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt'
pkl_file = 'batt_data.pkl'

with open(os.path.join(data_path, pkl_file), 'rb') as f:
    bat_data = pickle.load(f)

print(bat_data['b1c0']['cycles'].keys())

temp = [int(cycle) for cycle in bat_data['b1c0']['cycles'].keys()]
print(max(temp))

In [ ]:
data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt/' + 'batt_data.pkl'

In [ ]:
dataset = BattData('/Users/ivan_zorin/Documents/AIRI/data/batt/', data_name='batt_data.pkl', is_save_data=False)
print(dataset[0])

In [ ]:
batt_fade = dataset.capacity_fade_approximation()

In [ ]:
hi_approximator = PolyApproximator
batt_fade_approx = {}


In [ ]:
for batt_name, value in dataset.batt_summary.items():
    QC = value['QC']
    QD = value['QD']

    qc_approx = hi_approximator(degree=2).fit(np.arange(len(QC)), QC)
    qd_approx = hi_approximator(degree=2).fit(np.arange(len(QD)), QD)

    batt_fade_approx[batt_name] = {
                                    'QC': qc_approx,
                                    'QD': qd_approx
                                    }

In [ ]:
batt_fade_approx['b1c0']['QC']

In [ ]:
dataloader = BattDataloader(dataset, load_features=['I', 'T'], batch_size=1, shuffle=False)
batch = next(iter(dataloader))
print(batch)

In [ ]:
t = tqdm(dataloader)

for i, batch in enumerate(t):
    pass

In [ ]:
for len_ in range(200):
    # print(len_)
    dataset_min_len = len_
    min_idx = []

    for i, sample in enumerate(dataset):
        if dataset_min_len == sample[2]:
            # dataset_min_len = sample[2]
            min_idx += [i]

    if len(min_idx) > 0:
        print(len_)
        print(min_idx)  
        print(len(min_idx))

        check_batts = [dataset[idx][0] for idx in min_idx]
        batch_name = [name[:2] for name in check_batts]

        print(len(batch_name), set(batch_name))
        print('=======')

In [ ]:
data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt'


with open(os.path.join(data_path, 'batch1.pkl'), 'rb') as f:
    bat_data = pickle.load(f)

In [ ]:
len(bat_data.keys())

In [3]:
data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt/' # change the path 

# LSQ B-Spline approximation 
hi_approximator = SplineApproximator
hi_approximator_kwargs = {'degree': 3, 'knot_step': 100}
# dataset init
dataset = BattData(data_path, 'batt_data.pkl', hi_approximator=hi_approximator, hi_approximator_kwargs=hi_approximator_kwargs,  is_save_data=False)

data loaded from pickle file
start building dataset


In [6]:
batteries = dataset.batt_names

train_batts = [batt for batt in batteries if 'b1c' in batt]
test_batts = [batt for batt in batteries if 'b2c' in batt]

train_idxs = dataset.get_indices_by_batt(train_batts)
test_idxs = dataset.get_indices_by_batt(test_batts)

train_dataset = torch.utils.data.Subset(dataset, train_idxs)
test_dataset = torch.utils.data.Subset(dataset, test_idxs)


In [14]:
train_dataloader = torch.utils.data.DataLoader(train_dataset)

In [27]:
# __main__
data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt/' # change the path 


hi_approximator = SplineApproximator
hi_approximator_kwargs = {'degree': 3, 'knot_step': 100}
# dataset init
dataset = BattData(data_path, 'batt_data.pkl', hi_approximator=hi_approximator, hi_approximator_kwargs=hi_approximator_kwargs,  is_save_data=False)

# example of train-test split 
batteries = dataset.batt_names
train_batts = [batt for batt in batteries if 'b1c' in batt]
test_batts = [batt for batt in batteries if 'b2c' in batt]

train_idxs = dataset.get_indices_by_batt(train_batts)
test_idxs = dataset.get_indices_by_batt(test_batts)

train_dataset = torch.utils.data.Subset(dataset, train_idxs)
test_dataset = torch.utils.data.Subset(dataset, test_idxs)
# ==========


# dataloader init
shuffle = True
dataloader = BattDataloader(dataset, load_features=['I', 'V', 'T'], batch_size=2, shuffle=shuffle)


data loaded from pickle file
start building dataset


In [ ]:

batch = next(iter(dataloader))
batt_names, cycles, ruls, his_approx, reference_his, features = batch
print(f'''
Batch structure
    batteries: {batt_names},
    cycles: {cycles},
    RULs: {ruls},
    approximation of HIs: {his_approx},
    HI of reference battery: {reference_his}, 
    features shape: {features.shape}
''')

t = tqdm(dataloader)
for i, batch in enumerate(t):
    pass


In [38]:
train_dataloader = BattDataloader(train_dataset, load_features=['I', 'V', 'T'], batch_size=2, shuffle=shuffle)
test_dataloader = BattDataloader(test_dataset, load_features=['I', 'V', 'T'], batch_size=2, shuffle=shuffle)


In [35]:
batch = next(iter(train_dataloader))

In [39]:
batteries = []
for batch in test_dataloader:
    batteries += batch[0]


print(set([batt[:2] for batt in batteries]))

{'b2'}


In [46]:
len(train_dataloader), len(train_dataloader.dataloader)

(18847, 18847)

In [48]:
tsk_msk = 22292
housing = 6492
msk_tsk = 11823

total = tsk_msk + housing + msk_tsk
print(total)

40607


900